In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
sanjuan_dataset = pd.read_csv("../../dengueData/SanJuan/san_juan_training_data.csv")
print(sanjuan_dataset.head())
print("Number of observations: ", len(sanjuan_dataset))

      season  season_week week_start_date  denv1_cases  denv2_cases  \
0  1990/1991            1      1990-04-30            0            0   
1  1990/1991            2      1990-05-07            0            0   
2  1990/1991            3      1990-05-14            0            0   
3  1990/1991            4      1990-05-21            0            0   
4  1990/1991            5      1990-05-28            0            0   

   denv3_cases  denv4_cases  other_positive_cases  additional_cases  \
0            0            0                     4                 0   
1            0            0                     5                 0   
2            0            0                     4                 0   
3            0            0                     3                 0   
4            0            0                     6                 0   

   total_cases  
0            4  
1            5  
2            4  
3            3  
4            6  
Number of observations:  988


In [3]:
sanjuan_cases = sanjuan_dataset["total_cases"].values
print(sanjuan_cases.shape)

(988,)


In [4]:
split_index = int(len(sanjuan_cases)* 0.7)
window_size = 4
future_steps = 1
train = sanjuan_cases[:split_index+window_size]

#Test goes from train to end - future_steps (as this data will be feed into the model)
#Expected is always future_steps ahead of test as this is the expected predictions
test = sanjuan_cases[split_index+window_size-future_steps:-future_steps]
expected = sanjuan_cases[split_index+window_size:]

print(test[-5:])
print(expected[-5:])
print(train.shape)
print(test.shape)
print(expected.shape)
print(sanjuan_cases[-5:])

[26 24  9 21 10]
[24  9 21 10 15]
(695,)
(293,)
(293,)
[24  9 21 10 15]


## Visualize ACF 

In [13]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import time
from statsmodels.tsa.statespace.sarimax import SARIMAX
history = [x for x in train]
predictions = list()
start = time.time()
for t in range(len(test)):
#     model = ARIMA(history, order=(5,1,0))
    model = SARIMAX(
        endog = history,
        order=(0, 1, 2),
        seasonal_order=(2, 2, 1, 52),
        enforce_stationarity=False,
        enforce_invertibility=False)
    
    model_fit = model.fit()
    output = model_fit.forecast(steps=future_steps)
    yhat = output[0]
    predictions.append(yhat)
    obs = expected[t]
    history.append(test[t])
    print('predicted=%f, expected=%f' % (yhat, obs))

end = time.time()
print("Total time: ", end-start)
error = mean_squared_error(expected, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(expected)
plt.plot(predictions, color='red')
plt.show()

predicted=30.337509, expected=35.000000
predicted=15.206061, expected=28.000000
predicted=38.353072, expected=37.000000


KeyboardInterrupt: 

In [34]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(expected, predictions)
print(mae)

8.022327034045007
